In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from env import host, user, password

# visualize
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(11, 9))
plt.rc('font', size=13)

# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

import os
os.path.isfile('telco_df.csv')

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

import wrangle

In [2]:
def get_connection(db, user=user, host=host, password=password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

# Exercises I - Required
- Let's review the steps we take at the beginning of each new module.
- #1. Create a new repository named regression-exercises in your GitHub; all of your Regression work will be housed here.
- #2. Clone this repository within your local codeup-data-science directory.
- #3. Create a .gitignore and make sure your list of 'files to ignore' includes your env.py file.
- #4. Ceate a README.md file that outlines the contents and purpose of your repository.
- #5. Add, commit, and push these two files.
- #6. Now you can add your env.py file to this repository to access the Codeup database server.
- #7. For these exercises, you will create wrangle.ipynb and wrangle.py files to hold necessary functions.
- #8. As always, add, commit, and push your work often.


# Exercises II - Required
- Let's set up an example scenario as perspective for our regression exercises using the Telco dataset.

- As a customer analyst for Telco, you want to know who has spent the most money with the company over their lifetime. You have monthly charges and tenure, so you think you will be able to use those two attributes as features to estimate total charges. You need to do this within an average of $5.00 per customer.

- In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Telco data from the telco_churn database in the Codeup database server.


In [3]:
#1. Acquire customer_id, monthly_charges, tenure, and total_charges from the telco_churn database for all customers with a 2-year contract.

In [4]:
#acquire telco dataframe
telco_df= wrangle.get_telco_data()

In [5]:
telco_df.head()

payment_type_id  internet_service_type_id  contract_type_id customer_id  \
0                2                         1                 3  0016-QLJIS   
1                4                         1                 3  0017-DINOC   
2                3                         1                 3  0019-GFNTW   
3                4                         1                 3  0056-EPFBG   
4                3                         1                 3  0078-XZMHT   

   gender  senior_citizen partner dependents  tenure phone_service  ...  \
0  Female               0     Yes        Yes      65           Yes  ...   
1    Male               0      No         No      54            No  ...   
2  Female               0      No         No      56            No  ...   
3    Male               0     Yes        Yes      20            No  ...   
4    Male               0     Yes         No      72           Yes  ...   

  tech_support streaming_tv streaming_movies paperless_billing  \
0          Yes          Yes              Yes               Yes   
1          Yes          Yes               No                No   
2          Yes           No               No                No   
3          Yes           No               No               Yes   
4          Yes          Yes              Yes               Yes   

  monthly_charges total_charges churn contract_type  internet_service_type  \
0           90.45        5957.9    No      Two year                    DSL   
1           45.20       2460.55    No      Two year                    DSL   
2           45.05        2560.1    No      Two year                    DSL   
3           39.40         825.4    No      Two year                    DSL   
4           85.15        6316.2    No      Two year                    DSL   

                payment_type  
0               Mailed check  
1    Credit card (automatic)  
2  Bank transfer (automatic)  
3    Credit card (automatic)  
4  Bank transfer (automatic)  

[5 rows x 24 columns]

In [6]:
telco_df['contract_type_id'].value_counts() # Look at unique values in contract_type_id column

1    3875
3    1695
2    1473
Name: contract_type_id, dtype: int64

In [7]:

#creating sql query to filter columns to customer_id, monthly_charges, tenure, total_charges, and contract_type_id
two_year_query= " Select customer_id, monthly_charges, tenure, total_charges FROM customers  JOIN contract_types USING(contract_type_id)  JOIN internet_service_types USING(internet_service_type_id) JOIN payment_types USING(payment_type_id) WHERE `contract_type_id` = '3';"

In [8]:
#using pandas to read sql query above 
telco_two_year_df= pd.read_sql(two_year_query, get_connection('telco_churn'))

In [9]:
telco_two_year_df.head()

customer_id  monthly_charges  tenure total_charges
0  0016-QLJIS            90.45      65        5957.9
1  0017-DINOC            45.20      54       2460.55
2  0019-GFNTW            45.05      56        2560.1
3  0056-EPFBG            39.40      20         825.4
4  0078-XZMHT            85.15      72        6316.2

In [10]:
#2. Using your acquired Telco data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. 
# You may handle the missing values however you feel is appropriate and meaningful; 
# remember to document your process and decisions using markdown and code commenting where helpful.

In [11]:
telco_two_year_df.head()#Looking at first 5 values

customer_id  monthly_charges  tenure total_charges
0  0016-QLJIS            90.45      65        5957.9
1  0017-DINOC            45.20      54       2460.55
2  0019-GFNTW            45.05      56        2560.1
3  0056-EPFBG            39.40      20         825.4
4  0078-XZMHT            85.15      72        6316.2

In [12]:

telco_two_year_df.shape # Looking at shape: 1695 rows and 4 columns

(1695, 4)

In [13]:
telco_two_year_df.describe().T

count       mean        std   min     25%    50%    75%  \
monthly_charges  1695.0  60.770413  34.678865  18.4  24.025  64.35  90.45   
tenure           1695.0  56.735103  18.209363   0.0  48.000  64.00  71.00   

                    max  
monthly_charges  118.75  
tenure            72.00

**From the list of datatypes shown below we need to change total_charges from an object to a numeric datatype.**
- total_charges has odd values causing it to be a object instead of a numeric datatype

In [14]:
telco_two_year_df.info() #Looking at datatypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


**Below we can see there are no null values present in our current dataframe.**

In [15]:
telco_two_year_df.isnull().sum() # check to see if there are any null values

customer_id        0
monthly_charges    0
tenure             0
total_charges      0
dtype: int64

In [16]:
# Check for any Null values in each column of our DataFrame.

telco_two_year_df.isnull().any()

customer_id        False
monthly_charges    False
tenure             False
total_charges      False
dtype: bool

**We can see below there are 10 columns in the total charges column that are empty values**

In [17]:
telco_two_year_df[telco_two_year_df['total_charges'] == ' '] # Looking at which columns in total charges that is empty

customer_id  monthly_charges  tenure total_charges
85    1371-DWPAZ            56.05       0              
156   2775-SEFEE            61.90       0              
236   4075-WKNIU            73.35       0              
255   4472-LVYGI            52.55       0              
339   5709-LVOEQ            80.85       0              
1221  2520-SGTTA            20.00       0              
1257  3115-CZMZD            20.25       0              
1267  3213-VVOLG            25.35       0              
1338  4367-NUYAO            25.75       0              
1547  7644-OMVMY            19.85       0

**How do we handle the empty values?**
- Need to add zero to be able to convert to a numeric datatype b/c if you try to change an empty column to numeric it will produce an error.

In [18]:
#Add zero to columns to convert to float
telco_two_year_df['total_charges'] = telco_two_year_df['total_charges'] + '0'

In [19]:
#make total charges into datatype float
telco_two_year_df['total_charges'] = telco_two_year_df['total_charges'].astype('float')

In [20]:
telco_two_year_df.info() # making sure total_charges was changed to a float datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.1+ KB


In [21]:
#3. End with a wrangle.py file that contains the necessary functions to automate your process from 
# acquiring the data to returning a cleaned dataframe with no missing values. 
# Name your final function wrangle_telco.

**Making wrangle_telco function and making sure it works**

In [22]:
def new_telco_data():
    '''
    This function reads data from the Codeup db into a df.
    '''
    telco_sql = "SELECT * \
                 FROM customers \
                 JOIN contract_types USING(contract_type_id) \
                 JOIN internet_service_types USING(internet_service_type_id)\
                 JOIN payment_types USING(payment_type_id);"    
    return pd.read_sql(telco_sql, get_connection('telco_churn'))

In [23]:
def get_telco_data(cached=False):
    '''
    This function reads in telco churn data from Codeup database and writes data to
    a csv file if cached == False or if cached == True reads in telco df from
    a csv file, returns df.
    '''
    if cached == False or os.path.isfile('telco_df.csv') == False:
        
        # Read fresh data from db into a DataFrame.
        df = new_telco_data()
        
        # Write DataFrame to a csv file.
        df.to_csv('telco_df.csv')
        
    else:
        
        # If csv file exists or cached == True, read in data from csv.
        df = pd.read_csv('telco_df.csv', index_col=0)
        
    return df


In [24]:
def telco_two_year(df):
    query= "Select customer_id, monthly_charges, tenure, total_charges FROM customers JOIN contract_types USING(contract_type_id) JOIN internet_service_types USING(internet_service_type_id)JOIN payment_types USING(payment_type_id) WHERE `contract_type_id` = '3';"
    df= pd.read_sql(query, get_connection('telco_churn'))
    return df

In [25]:
def clean_data(df):
    '''
    This function take in the telco dataframe created and clean the total charges column
    by add a 0 to all empty columns and change the column to a float datatype.
    Returns new dataframe with total charges column cleaned.
    '''
    #Add zero to columns to convert to float
    df['total_charges'] = df['total_charges'] + '0'
    #make total charges into datatype float
    df['total_charges'] = df['total_charges'].astype('float')
    return df
    

In [26]:
def telco_split(df):
    '''
    This function take in the telco data acquired by get_telco_data,
    performs a split.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123)
    return train, validate, test


**Created wrangle_telco function and used it**

In [27]:
def wrangle_telco(df):
    df = clean_data(get_telco_data())
    return telco_split (df)

In [28]:
#acquire telco dataframe
telco_copy= wrangle.get_telco_data()

In [29]:
#used new wrangle_telco function to make sure it worked
train,validate, test= wrangle_telco(telco_df)

In [35]:
train

payment_type_id  internet_service_type_id  contract_type_id customer_id  \
4604                3                         2                 1  5564-NEMQO   
5566                4                         3                 3  0825-CPPQH   
6204                2                         3                 2  1561-BWHIN   
5837                3                         3                 3  4979-HPRFL   
1276                1                         1                 1  0749-IRGQE   
...               ...                       ...               ...         ...   
1794                2                         1                 1  4827-USJHP   
817                 2                         1                 2  3417-TSCIC   
901                 4                         1                 2  4825-XJGDM   
1987                1                         1                 1  6410-LEFEN   
6958                3                         3                 1  8204-TIFGJ   

      gender  senior_citizen partner dependents  tenure phone_service  ...  \
4604  Female               1      No         No       1           Yes  ...   
5566  Female               0     Yes         No      71           Yes  ...   
6204    Male               0     Yes        Yes      19           Yes  ...   
5837    Male               0     Yes        Yes      56           Yes  ...   
1276  Female               1     Yes         No      13            No  ...   
...      ...             ...     ...        ...     ...           ...  ...   
1794    Male               0      No         No      20           Yes  ...   
817     Male               0      No         No      29            No  ...   
901   Female               0      No         No      61            No  ...   
1987  Female               0      No         No       9           Yes  ...   
6958  Female               0      No         No      23           Yes  ...   

             tech_support         streaming_tv     streaming_movies  \
4604                   No                   No                   No   
5566  No internet service  No internet service  No internet service   
6204  No internet service  No internet service  No internet service   
5837  No internet service  No internet service  No internet service   
1276                   No                  Yes                  Yes   
...                   ...                  ...                  ...   
1794                   No                   No                   No   
817                    No                   No                   No   
901                   Yes                   No                   No   
1987                   No                   No                   No   
6958  No internet service  No internet service  No internet service   

     paperless_billing monthly_charges total_charges churn   contract_type  \
4604               Yes           75.30         75.30   Yes  Month-to-month   
5566                No           19.10       1372.45    No        Two year   
6204                No           19.80        344.50    No        One year   
5837                No           24.15       1402.25    No        Two year   
1276                No           45.30        528.45    No  Month-to-month   
...                ...             ...           ...   ...             ...   
1794               Yes           51.80       1023.85    No  Month-to-month   
817                Yes           24.85        788.05    No        One year   
901                 No           43.70       2696.55    No        One year   
1987                No           45.15        416.45   Yes  Month-to-month   
6958               Yes           20.30        470.60    No  Month-to-month   

      internet_service_type               payment_type  
4604            Fiber optic  Bank transfer (automatic)  
5566                   None    Credit card (automatic)  
6204                   None               Mailed check  
5837                   None  Bank transfer (automatic)  
1276                 

In [36]:
validate

payment_type_id  internet_service_type_id  contract_type_id customer_id  \
5854                3                         3                 3  5220-AGAAX   
797                 1                         1                 2  2988-PLAHS   
3188                4                         2                 2  6202-DYYFX   
2254                1                         1                 1  8614-VGMMV   
5732                2                         3                 3  3279-DYZQM   
...               ...                       ...               ...         ...   
1626                3                         1                 1  3640-JQGJG   
3674                1                         2                 1  1379-FRVEB   
1970                2                         1                 1  6260-ONULR   
6030                2                         3                 3  8043-PNYSD   
3202                3                         2                 2  6408-OTUBZ   

      gender  senior_citizen partner dependents  tenure phone_service  ...  \
5854    Male               0     Yes        Yes      68           Yes  ...   
797   Female               0      No         No       3           Yes  ...   
3188  Female               0      No         No      22           Yes  ...   
2254  Female               0      No         No      15           Yes  ...   
5732    Male               0     Yes        Yes      71           Yes  ...   
...      ...             ...     ...        ...     ...           ...  ...   
1626    Male               0     Yes        Yes      13           Yes  ...   
3674    Male               0      No        Yes      15           Yes  ...   
1970    Male               0      No         No       1           Yes  ...   
6030    Male               0     Yes        Yes      63           Yes  ...   
3202  Female               0      No         No      66           Yes  ...   

             tech_support         streaming_tv     streaming_movies  \
5854  No internet service  No internet service  No internet service   
797                    No                  Yes                  Yes   
3188                   No                   No                   No   
2254                   No                   No                   No   
5732  No internet service  No internet service  No internet service   
...                   ...                  ...                  ...   
1626                   No                   No                   No   
3674                   No                  Yes                   No   
1970                   No                  Yes                  Yes   
6030  No internet service  No internet service  No internet service   
3202                  Yes                  Yes                   No   

     paperless_billing monthly_charges total_charges churn   contract_type  \
5854               Yes           24.00       1664.30    No        Two year   
797                Yes           69.95        220.45    No        One year   
3188               Yes           76.00       1783.60    No        One year   
2254               Yes           49.10        679.55   Yes  Month-to-month   
5732                No           19.45       1378.45    No        Two year   
...                ...             ...           ...   ...             ...   
1626                No           44.80        559.20    No  Month-to-month   
3674               Yes           91.00       1430.05    No  Month-to-month   
1970               Yes           62.80         62.80    No  Month-to-month   
6030                No           19.55       1245.60    No        Two year   
3202               Yes          104.55       6779.05    No        One year   

      internet_service_type               payment_type  
5854                   None  Bank transfer (automatic)  
797                     DSL           Electronic check  
3188            Fiber optic    Credit card (automatic)  
2254                    DSL           Electronic check  
5732                 

In [37]:
test

payment_type_id  internet_service_type_id  contract_type_id customer_id  \
941                 4                         1                 2  5701-GUXDC   
1404                4                         1                 1  1794-HBQTJ   
5515                2                         2                 1  9985-MWVIX   
3684                1                         2                 1  1427-VERSM   
7017                2                         3                 1  9391-TTOYH   
...               ...                       ...               ...         ...   
2464                2                         2                 3  1063-DHQJF   
5585                4                         3                 3  1093-YSWCA   
804                 2                         1                 2  3137-NYQQI   
3190                4                         2                 2  6242-SGYTS   
3266                4                         2                 2  7632-MNYOY   

      gender  senior_citizen partner dependents  tenure phone_service  ...  \
941   Female               0     Yes         No      26            No  ...   
1404  Female               0      No         No       1           Yes  ...   
5515  Female               0      No         No       1           Yes  ...   
3684  Female               0     Yes         No      56           Yes  ...   
7017  Female               0      No         No      23           Yes  ...   
...      ...             ...     ...        ...     ...           ...  ...   
2464    Male               0     Yes        Yes      69           Yes  ...   
5585    Male               0      No         No      11           Yes  ...   
804     Male               0     Yes         No      17           Yes  ...   
3190    Male               0     Yes        Yes      62           Yes  ...   
3266    Male               1      No         No      66           Yes  ...   

             tech_support         streaming_tv     streaming_movies  \
941                    No                   No                   No   
1404                   No                   No                   No   
5515                   No                   No                   No   
3684                  Yes                  Yes                  Yes   
7017  No internet service  No internet service  No internet service   
...                   ...                  ...                  ...   
2464                  Yes                  Yes                   No   
5585  No internet service  No internet service  No internet service   
804                    No                   No                  Yes   
3190                   No                  Yes                   No   
3266                  Yes                  Yes                  Yes   

     paperless_billing monthly_charges total_charges churn   contract_type  \
941                 No           35.40        978.60    No        One year   
1404               Yes           48.60         48.60   Yes  Month-to-month   
5515               Yes           70.15         70.15   Yes  Month-to-month   
3684               Yes           98.70       5669.50    No  Month-to-month   
7017               Yes           19.50        470.20    No  Month-to-month   
...                ...             ...           ...   ...             ...   
2464               Yes           92.15       6480.90    No        Two year   
5585                No           19.55        223.15    No        Two year   
804                 No           64.80       1175.60    No        One year   
3190               Yes           94.95       5791.85    No        One year   
3266                No          110.90       7432.05   Yes        One year   

      internet_service_type             payment_type  
941                     DSL  Credit card (automatic)  
1404                    DSL  Credit card (automatic)  
5515            Fiber optic             Mailed check  
3684            Fiber optic         Electronic check  
7017                   None    

# Exercises III - Challenge
- Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

- As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.

- In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

In [31]:
# def get_zillow_data():
#     '''
#     This function reads in Zillow data from CodeUp db and creates a dataframe
#     '''
#     zillow_query = '''SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips 
#     FROM properties_2016 
#     WHERE propertylandusetypeid = 261;'''
#     return pd.read_sql(zillow_query, get_connection('zillow'))

In [2]:
zillow_df= wrangle.get_zillow_data()

In [3]:
zillow_df.head()

payment_type_id  internet_service_type_id  contract_type_id customer_id  \
0                2                         1                 3  0016-QLJIS   
1                4                         1                 3  0017-DINOC   
2                3                         1                 3  0019-GFNTW   
3                4                         1                 3  0056-EPFBG   
4                3                         1                 3  0078-XZMHT   

   gender  senior_citizen partner dependents  tenure phone_service  ...  \
0  Female               0     Yes        Yes      65           Yes  ...   
1    Male               0      No         No      54            No  ...   
2  Female               0      No         No      56            No  ...   
3    Male               0     Yes        Yes      20            No  ...   
4    Male               0     Yes         No      72           Yes  ...   

  tech_support streaming_tv streaming_movies paperless_billing  \
0          Yes          Yes              Yes               Yes   
1          Yes          Yes               No                No   
2          Yes           No               No                No   
3          Yes           No               No               Yes   
4          Yes          Yes              Yes               Yes   

  monthly_charges total_charges churn contract_type  internet_service_type  \
0           90.45        5957.9    No      Two year                    DSL   
1           45.20       2460.55    No      Two year                    DSL   
2           45.05        2560.1    No      Two year                    DSL   
3           39.40         825.4    No      Two year                    DSL   
4           85.15        6316.2    No      Two year                    DSL   

                payment_type  
0               Mailed check  
1    Credit card (automatic)  
2  Bank transfer (automatic)  
3    Credit card (automatic)  
4  Bank transfer (automatic)  

[5 rows x 24 columns]

In [4]:
#2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.

In [5]:
# Dataframe has 2152863 rows and 7 columns
zillow_df.shape

(7043, 24)

In [6]:
zillow_df.describe().T

count       mean        std    min   25%    50%  \
payment_type_id           7043.0   2.315633   1.148907   1.00   1.0   2.00   
internet_service_type_id  7043.0   1.872923   0.737796   1.00   1.0   2.00   
contract_type_id          7043.0   1.690473   0.833755   1.00   1.0   1.00   
senior_citizen            7043.0   0.162147   0.368612   0.00   0.0   0.00   
tenure                    7043.0  32.371149  24.559481   0.00   9.0  29.00   
monthly_charges           7043.0  64.761692  30.090047  18.25  35.5  70.35   

                            75%     max  
payment_type_id            3.00    4.00  
internet_service_type_id   2.00    3.00  
contract_type_id           2.00    3.00  
senior_citizen             0.00    1.00  
tenure                    55.00   72.00  
monthly_charges           89.85  118.75

In [7]:
zillow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

In [8]:
#Looking at how many null values
zillow_df.isnull().sum()

payment_type_id             0
internet_service_type_id    0
contract_type_id            0
customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
contract_type               0
internet_service_type       0
payment_type                0
dtype: int64

In [9]:
#Looking at how many null values (boolean)
zillow_df.isnull().any()

payment_type_id             False
internet_service_type_id    False
contract_type_id            False
customer_id                 False
gender                      False
senior_citizen              False
partner                     False
dependents                  False
tenure                      False
phone_service               False
multiple_lines              False
online_security             False
online_backup               False
device_protection           False
tech_support                False
streaming_tv                False
streaming_movies            False
paperless_billing           False
monthly_charges             False
total_charges               False
churn                       False
contract_type               False
internet_service_type       False
payment_type                False
dtype: bool

In [10]:
# Replace white space values with NaN values.
zillow_df= zillow_df.replace(r'^\s*$', np.nan, regex=True)


In [11]:
#Looking at how many null values
zillow_df.isnull().sum()

payment_type_id              0
internet_service_type_id     0
contract_type_id             0
customer_id                  0
gender                       0
senior_citizen               0
partner                      0
dependents                   0
tenure                       0
phone_service                0
multiple_lines               0
online_security              0
online_backup                0
device_protection            0
tech_support                 0
streaming_tv                 0
streaming_movies             0
paperless_billing            0
monthly_charges              0
total_charges               11
churn                        0
contract_type                0
internet_service_type        0
payment_type                 0
dtype: int64

In [12]:
 # Drop all rows with NaN values.
zillow_df= zillow_df.dropna()

In [13]:
#Looking at how many null values
zillow_df.isnull().sum()

payment_type_id             0
internet_service_type_id    0
contract_type_id            0
customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
contract_type               0
internet_service_type       0
payment_type                0
dtype: int64

In [14]:
#3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe witn no missing values in your wrangle.py file. Name your final function wrangle_zillow.

In [15]:
zillow_copy= wrangle.get_zillow_data()

In [16]:
wrangle.wrangle_zillow(zillow_copy)

payment_type_id  internet_service_type_id  contract_type_id customer_id  \
0                   2                         1                 3  0016-QLJIS   
1                   4                         1                 3  0017-DINOC   
2                   3                         1                 3  0019-GFNTW   
3                   4                         1                 3  0056-EPFBG   
4                   3                         1                 3  0078-XZMHT   
...               ...                       ...               ...         ...   
7038                2                         3                 1  9962-BFPDU   
7039                2                         3                 1  9967-ATRFS   
7040                4                         3                 1  9970-QBCDA   
7041                2                         3                 1  9975-SKRNR   
7042                3                         3                 1  9986-BONCE   

      gender  senior_citizen partner dependents  tenure phone_service  ...  \
0     Female               0     Yes        Yes      65           Yes  ...   
1       Male               0      No         No      54            No  ...   
2     Female               0      No         No      56            No  ...   
3       Male               0     Yes        Yes      20            No  ...   
4       Male               0     Yes         No      72           Yes  ...   
...      ...             ...     ...        ...     ...           ...  ...   
7038  Female               0     Yes        Yes       1           Yes  ...   
7039  Female               0      No         No      19           Yes  ...   
7040  Female               0      No         No       6           Yes  ...   
7041    Male               0      No         No       1           Yes  ...   
7042  Female               0      No         No       4           Yes  ...   

             tech_support         streaming_tv     streaming_movies  \
0                     Yes                  Yes                  Yes   
1                     Yes                  Yes                   No   
2                     Yes                   No                   No   
3                     Yes                   No                   No   
4                     Yes                  Yes                  Yes   
...                   ...                  ...                  ...   
7038  No internet service  No internet service  No internet service   
7039  No internet service  No internet service  No internet service   
7040  No internet service  No internet service  No internet service   
7041  No internet service  No internet service  No internet service   
7042  No internet service  No internet service  No internet service   

     paperless_billing monthly_charges total_charges churn   contract_type  \
0                  Yes           90.45        5957.9    No        Two year   
1                   No           45.20       2460.55    No        Two year   
2                   No           45.05        2560.1    No        Two year   
3                  Yes           39.40         825.4    No        Two year   
4                  Yes           85.15        6316.2    No        Two year   
...                ...             ...           ...   ...             ...   
7038                No           20.05         20.05    No  Month-to-month   
7039                No           19.90        367.55    No  Month-to-month   
7040                No           19.70        129.55    No  Month-to-month   
7041                No           18.90          18.9    No  Month-to-month   
7042                No           20.95          85.5   Yes  Month-to-month   

      internet_service_type               payment_type  
0                       DSL               Mailed check  
1                       DSL    Credit card (automatic)  
2                       DSL  Bank transfer (automatic)  
3                       DSL    Credit card (automatic)  
4                    

In [17]:
#Looking at how many null values to verify all nulls were dropped
zillow_copy.isnull().sum()

payment_type_id             0
internet_service_type_id    0
contract_type_id            0
customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
contract_type               0
internet_service_type       0
payment_type                0
dtype: int64